In [1]:
#February 21, 2020
import os
import pandas as pd
#home_folder = os.path.expanduser("~")
data_folder = os.path.join(os.path.expanduser("~"), "OneDrive", "Desktop", "Data Mining", "Chapter4HW", "Data", "ml-100k") #home_folder
ratings_filename = os.path.join(data_folder, "u.data")

In [2]:
#Modify data by setting delimiter to tab character, tell pandas not to read the first row as the header, and instead set column names with heading values
all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"]) 

In [3]:
#Display the first few records in the data
all_ratings.head()

,UserID,MovieID,Rating,Datetime
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
#Used to parse date timestamp, uses as important feature in recommendation prediction, movies rated together have more similar ranks than those separate, can improve quality of data
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'], unit='s')

In [5]:
#Displays the first few records in the data, with proper updated timestamp format
all_ratings.head()

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [6]:
#Rule: If a person recommends this set of movies, they will also recommend this movie
#Creates new variable called "Favorable" which holds "True" if the person gave a favorable review to a movie.
#Anything over 3 stars is considered favorable
#View Feature By Using: all_ratings[10:15] (optional)
all_ratings["Favorable"] = all_ratings["Rating"] > 3

In [7]:
#Sample Datasets to form training data, helps reduce size of dataset to make Apriori Algorithm run faster
#Locates all ratings/ reviews from the first 200 users 
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]

In [8]:
#Sets the favorable_ratings as a variable to be used within the program for convenience in making the dataset
#Performs in one line: favorable_ratings_mask = ratings["Favorable"] favorable_ratings = ratings[favorable_ratings_mask] 
favorable_ratings = ratings[ratings["Favorable"]]

In [9]:
#Obtains the movies that each user has given a favorable rating by grouping the dataset based on UserID and iterate over movies in each group
#Stored values in frozenset to quickly verify if a movie has been rated by user
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("UserID")["MovieID"])

In [10]:
#Create DataFrame to tell us how often/frequent a movie has been given a favorable review 
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()

In [11]:
#View the top five movies list by ID
num_favorable_by_movie.sort_values(by="Favorable", ascending=False).head()

,Favorable
MovieID,
50,100.0
100,89.0
258,83.0
181,79.0
174,74.0


In [12]:
#Stores the discovered frequent itemsets , key = length of itemsets, so that we can quickly access itemsets of a given length, namely recently discovered frequent itemsets
frequent_itemsets = {}

In [13]:
#Defines minimum support for itemset to be considered frequent, based on dataset, can be changed, but must keep in mind runtime
min_support = 50

In [14]:
#Used to implement the first step of Apriori Algorithm, performs faster set-based operation and used as keys in counting dictionary
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                            for movie_id, row in num_favorable_by_movie.iterrows()
                            if row["Favorable"] > min_support)

In [15]:
#Implements 2nd and 3rd steps efficiency , creates function that takes newly discovered frequent itemsets, creates super sets and tests the frequency of the sets
#Iterate over data set once per call, Single-Pass, iterate over each user and previously discovered itemsets
#Checks if subset of the current set of reviews and stores them in k_1_itemsets, if so the user reivewed each movie in itemset as shown in itemset.issubset(reviews)
from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [16]:
#Go through each individual movie the user has reviewed but not in itemset, creates superset by combining itemset with new movie and record/ counting dictionary
#Uses min_support value to test the function and returns values that are over that amount to be considered frequent
#Creates Loop that iterates over steps of the algorithm, stores itemsets as k increases in size, 
#K is length of discovered frequent itemsets, use k-1 key to view and create frequent itemset, stores them in dictionary by length
import sys #needed to use flush 
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 50

# k=1 candidates are the isbns with more than min_support favorable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)

print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those that are unnecessary
del frequent_itemsets[1]

There are 16 movies with more than 50 favorable reviews
I found 93 frequent itemsets of length 2
I found 295 frequent itemsets of length 3
I found 593 frequent itemsets of length 4
I found 785 frequent itemsets of length 5
I found 677 frequent itemsets of length 6
I found 373 frequent itemsets of length 7
I found 126 frequent itemsets of length 8
I found 24 frequent itemsets of length 9
I found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11


In [17]:
#Association Rule: "If a reviewer recommends all of the movies in the premise, they will also recommend the conclusion movie"
#Takes movie in itemset and denotes it as conclusion, the rest serve as the premise
#Generate a List of All Rules, in Frequent Itemsets and iterate over discovered frequent itemsets, and each movie in itemset
# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules))) 

There are 15285 candidate rules


In [18]:
#Prints out first five rules in list, the first parameter, frozenset shows the list of movies in the premise, number afterwards is the conclusion
#If a reviewer recommends movie 7, they are also likely to recommend movie 1, the opposite is also true as we see in the second case
print(candidate_rules[:5])

[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56)]


In [19]:
#Compute the Confidence, creates dictionaries to store the numbe of occurances we see the premise leading to conclucion, and how many times it doesn't and iterate over cases where premise of rules apply
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules} #Divide the correct count by number of time rule was seen

In [20]:
# Choose only rules above a minimum confidence level, chosen to be less than 1
min_confidence = 0.9

In [21]:
# Filter out the rules with poor confidence so that they don't interfere with the useful data
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

5152


In [22]:
#Sorts the confidence dictionary
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

In [23]:
#Prints results of sorting
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({98, 181}) they will also recommend 50
 - Confidence: 1.000

Rule #2
Rule: If a person recommends frozenset({172, 79}) they will also recommend 174
 - Confidence: 1.000

Rule #3
Rule: If a person recommends frozenset({258, 172}) they will also recommend 174
 - Confidence: 1.000

Rule #4
Rule: If a person recommends frozenset({1, 181, 7}) they will also recommend 50
 - Confidence: 1.000

Rule #5
Rule: If a person recommends frozenset({1, 172, 7}) they will also recommend 174
 - Confidence: 1.000



In [24]:
#Loads the titles from the u.items file with movie names and MovieID
#Other information is found in README file, CSV fromat but not header, use column names
# Even better, we can get the movie titles themselves from the dataset
movie_name_filename = os.path.join(data_folder, "u.item")
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")
movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [25]:
#Prints out this data
movie_name_data.head()

,MovieID,Title,Release Date,Video Release,IMDB,<UNK>,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [26]:
#Creates function to get the movie title from the MovieID, so that we dont look it up each time
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title

In [27]:
#Prints out a movie title using the above function
get_movie_name(4)

'Get Shorty (1995)'

In [28]:
#Adjusts the previous function to print out the top rules with the titles
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence: 1.000

Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #3
Rule: If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #4
Rule: If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Confidence: 1.000

Rule #5
Rule: If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000



In [29]:
#Evaluate the association rules using test data that was not used for training and evaluate the rules, computes test set confidence 
#Extract the test dataset, all records we didnt use in training (first 200 users), get all favorable reviews
#Evaluation using test data
test_dataset = all_ratings[~all_ratings['UserID'].isin(range(200))]
test_favorable = test_dataset[test_dataset["Favorable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])

In [30]:
#Count the correct instances where the premise leads to the conclusion, as before with the previous training data, here we use test data
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [31]:
#Compute the confidence from the correct counts and sorts them
test_confidence = {candidate_rule:
                   (correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule]))
                   for candidate_rule in rule_confidence}
print(len(test_confidence)) #sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)

5152


In [32]:
#Print out the best association rules with titles instead of MovieIDs
for index in range(10):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Train Confidence: 1.000
 - Test Confidence: 0.936

Rule #2
Rule: If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.876

Rule #3
Rule: If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.841

Rule #4
Rule: If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Train Confidence: 1.000
 - Test Confidence: 0.932

Rule #5
Rule: If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence